# 결과 출력 코드

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/best_model.zip

Archive:  /content/drive/MyDrive/best_model.zip
  inflating: model/config.json       
  inflating: model/pytorch_model.bin  
  inflating: model/special_tokens_map.json  
  inflating: model/tokenizer.json    
  inflating: model/tokenizer_config.json  
  inflating: model/vocab.txt         
  inflating: KLUE_custom.zip         


In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 12.0 MB/s 
     |████████████████████████████████| 6.5 MB 39.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 37.1 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 12.2 MB/s 
     |████████████████████████████████| 134 kB 48.8 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
     |████████████████████████████████| 271 kB 48.3 MB/s 
     |████████████████████████████████| 144 kB 51.7 MB/s 
     |████████████████████████████████| 94 kB 1.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [4]:
import tarfile

#훈련 데이터셋 다운로드 및 압축풀기

!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

tar_bz2_file = tarfile.open("/content/klue-sts-v1.1.tar.gz")
tar_bz2_file.extractall(path="/content")
tar_bz2_file.close()

--2022-03-21 01:15:51--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.218.168.131
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.218.168.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  1.38MB/s    in 0.9s    

2022-03-21 01:15:53 (1.38 MB/s) - ‘klue-sts-v1.1.tar.gz’ saved [1349881/1349881]



In [5]:
import torch

#GPU 사용 여부 확인 및 name 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [6]:
#모델 custom을 진행한 폴더 압축해제
!unzip /content/KLUE_custom.zip

Archive:  /content/KLUE_custom.zip
  inflating: sts/__pycache__/dataset.cpython-37.pyc  
  inflating: sts/__pycache__/metric.cpython-37.pyc  
  inflating: sts/__pycache__/model.cpython-37.pyc  
  inflating: sts/__pycache__/utils.cpython-37.pyc  
  inflating: sts/data/download.sh    
  inflating: sts/README.md           
  inflating: sts/dataloader.py       
  inflating: sts/dataset.py          
  inflating: sts/infer_test.ipynb    
  inflating: sts/inference.py        
  inflating: sts/metric.py           
  inflating: sts/model.py            
 extracting: sts/requirements.txt    
  inflating: sts/train.py            
  inflating: sts/utils.py            


In [7]:
#학습한 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [11]:
import json
import pandas as pd

def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

In [27]:
#예측을 진행한 정보와 guid를 매칭

threshold=3.22

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')

df_pred = pd.read_csv('/content/output.csv',header=None)
df_pred.columns=['predict_real_label']
df_pred['guid']=[data['guid'] for data in valid_data]
df_pred=df_pred[['guid','predict_real_label']]
df_pred['predict_binary_label']=df_pred['predict_real_label']>threshold
df_pred['predict_binary_label']=df_pred['predict_binary_label'].astype(int)
display(df_pred.head())
print("shape:",df_pred.shape)

,guid,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,5.433781,1
1,klue-sts-v1_dev_00001,2.246123,0
2,klue-sts-v1_dev_00002,1.090579,0
3,klue-sts-v1_dev_00003,4.480976,1
4,klue-sts-v1_dev_00004,2.407049,0


shape: (519, 3)


In [33]:
dev_set_score=pd.read_csv('/content/dev_set_score.csv')
display(dev_set_score.head())
print("shape:",dev_set_score.shape)

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,NaN,NaN
1,klue-sts-v1_dev_00001,1.428571,0,NaN,NaN
2,klue-sts-v1_dev_00002,1.285714,0,NaN,NaN
3,klue-sts-v1_dev_00003,3.714286,1,NaN,NaN
4,klue-sts-v1_dev_00004,2.500000,0,NaN,NaN


shape: (519, 5)


In [34]:
dev_set_score_merge=pd.merge(dev_set_score.dropna(axis=1),df_pred,on='guid')
dev_set_score_merge

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,5.433781,1
1,klue-sts-v1_dev_00001,1.428571,0,2.246123,0
2,klue-sts-v1_dev_00002,1.285714,0,1.090579,0
3,klue-sts-v1_dev_00003,3.714286,1,4.480976,1
4,klue-sts-v1_dev_00004,2.500000,0,2.407049,0
...,...,...,...,...,...
514,klue-sts-v1_dev_00514,2.200000,0,4.327172,1
515,klue-sts-v1_dev_00515,2.833333,0,3.499312,1
516,klue-sts-v1_dev_00516,0.333333,0,2.349673,0
517,klue-sts-v1_dev_00517,0.333333,0,0.518402,0


In [40]:
dev_set_score_merge.to_csv("/content/dev_set_score.csv",index=False)

# F1 score 와 Pearson's r

In [37]:
from sklearn.metrics import classification_report,f1_score
import pandas as pd

#valid의 label 값을 추출
file_path="/content/dev_set_score.csv"

df=pd.read_csv(file_path)

#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화
print(classification_report(df['true_binary_label'], df['predict_binary_label']))
print('base_line(roberta-base) f1_score:',f1_score(df['true_binary_label'], df['predict_binary_label']),end='\n\n')

              precision    recall  f1-score   support

           0       0.97      0.82      0.89       299
           1       0.79      0.96      0.87       220

    accuracy                           0.88       519
   macro avg       0.88      0.89      0.88       519
weighted avg       0.89      0.88      0.88       519

base_line(roberta-base) f1_score: 0.8706365503080084



In [39]:
from datasets import load_metric

#dev_set에 대한 pearsonr score 출력 
pearson = load_metric("pearsonr").compute
pearson(predictions=df['predict_real_label'], references=df['true_real_label'])

{'pearsonr': 0.9020511636828425}